# Projekt Sztucznej Sieci Neuronowej
## By Szymon Skrzypczyk

Jest to projekt prostej sieci neuronowej, która będzie się uczyć na funkcji liniowej(z możliwością zmiany, dzięki możliwości dodania wielu ukrytych warstw).

In [74]:
%use lets-plot
import kotlin.random.Random
import org.jetbrains.letsPlot.ggplot
import org.jetbrains.letsPlot.geom.geomLine
import org.jetbrains.letsPlot.label.ggtitle
import org.jetbrains.letsPlot.gggrid

### Definicja Neuronu

In [101]:
import kotlin.math.exp
import kotlin.random.Random

fun sigmoid(x: Double): Double {
    return 1.0 / (1.0 + exp(-x))
}

fun sumWeights(input: MutableList<Double>, weights: MutableList<Double>): Double {
    var sum = weights[0]
    for (i in 1..<input.size) {
        sum += input[i] * weights[i]
    }
    return sum
}


class Neuron(
    var input: MutableList<Double>,
    var activationFunction: (Double) -> Double = ::sigmoid,
) {
    var output: Double = 0.0
    var error: Double = 0.0
    var weights: MutableList<Double> = MutableList(input.size) { 0.0 }

    init {
        // Since it is a creation of a neuron, wights will have random values
        for (i in weights.indices) {
            weights[i] = Random.nextDouble(-1.0, 1.0)
        }
    }

    fun feedForward() {
        val weightedSum = sumWeights(input, weights)
        output = activationFunction(weightedSum)
    }

    fun calculateError(target: Double) {
        // error as the difference between target and output
        error = target - output
    }

    fun updateWeights(learningRate: Double) {
        // each weight is updated by the product of learning rate, error and input
        for (i in weights.indices) {
            weights[i] += learningRate * error * input[i]
        }
    }
}

## Definicja Sieci Neuronowej

In [89]:
class NeuralNetwork(
    private val inputSize: Int,
    private val hiddenLayers: List<Int>,
    private val outputSize: Int,
    private val activationFunction: (Double) -> Double = ::sigmoid
) {
    private val layers: MutableList<MutableList<Neuron>> = mutableListOf()
    val trainErrors: MutableList<Double> = mutableListOf()
    var validationErrors: MutableList<Double> = mutableListOf()

    init {
        var previousSize = inputSize
        for (layerSize in hiddenLayers) {
            val layer = MutableList(layerSize) {
                Neuron(MutableList(previousSize) { 0.0 }, activationFunction)
            }
            layers.add(layer)
            previousSize = layerSize
        }
        val outputLayer = MutableList(outputSize) {
            Neuron(MutableList(previousSize) { 0.0 }, activationFunction)
        }
        layers.add(outputLayer)
    }

    fun feedForward(input: MutableList<Double>): MutableList<Double> {
        var currentInput = input
        for (layer in layers) {
            for (neuron in layer) {
                neuron.input = currentInput
                neuron.feedForward()
            }
            currentInput = layer.map { it.output }.toMutableList()
        }
        return currentInput
    }

    fun backpropagate(target: MutableList<Double>, learningRate: Double) {
        val outputLayer = layers.last()

        for (i in outputLayer.indices) {
            outputLayer[i].calculateError(target[i])
        }

        for (i in layers.size - 2 downTo 0) {
            val currentLayer = layers[i]
            val nextLayer = layers[i + 1]

            for (j in currentLayer.indices) {
                val neuron = currentLayer[j]
                val sum = nextLayer.sumOf { nextNeuron ->
                    if (j + 1 < nextNeuron.weights.size) {
                        nextNeuron.error * nextNeuron.weights[j + 1]
                    } else 0.0
                }
                neuron.error = sum * neuron.output * (1 - neuron.output)
            }
        }

        for (layer in layers) {
            for (neuron in layer) {
                neuron.updateWeights(learningRate)
            }
        }
    }

    fun train(
        inputs: List<MutableList<Double>>,
        targets: List<MutableList<Double>>,
        epochs: Int, learningRate: Double,
        validationInputs: List<MutableList<Double>>? = null,
        validationTargets: List<MutableList<Double>>? = null) {
        for (epoch in 0 until epochs) {
            for (i in inputs.indices) {
                feedForward(inputs[i])
                backpropagate(targets[i], learningRate)
                val output = layers.last().map { it.output }
                val error = Utils.MeanSquaredError(targets[i], output as MutableList<Double>)
                trainErrors.add(error)
            }

            if (validationInputs != null && validationTargets != null) {
                var valMSE = 0.0
                for (i in validationInputs.indices) {
                    val output = feedForward(validationInputs[i])
                    val error = Utils.MeanSquaredError(validationTargets[i], output)
                    valMSE += error
                }
                // divided to keep it consistent with training error
                validationErrors.add(valMSE / validationInputs.size)
            }
        }
    }

    fun predict(input: MutableList<Double>): MutableList<Double> {
        return feedForward(input)
    }
}

In [90]:
object Utils {
    fun MeanSquaredError(target: MutableList<Double>, predicted: MutableList<Double>): Double {
        var sum = 0.0
        for (i in target.indices) {
            sum += (target[i] - predicted[i]).pow(2)
        }
        return sum / target.size
    }
}

### Zdefiniowanie funkcji do uczenia

In [91]:
val step = 12.0 / 499
val x = generateSequence(0.0) { it + step }
    .takeWhile { it <= 12 }
    .toMutableList()
val y = x.map { 0.5 * cos(0.2 * it * it) + 0.5}.toMutableList()


In [92]:
val data = mapOf<String, Any>(
    "x" to x,
    "y" to y
)

val plot = ggplot(data) +
        geomLine(color = "red") { x = "x"; y = "y" } +
        ggtitle("y = 0.5 * cos(0.2 * x²) + 0.5")

plot

<path d="M24.407513580896307 14.590909090909122 L24.407513580896307 14.590909090909122 L25.38577063824687 14.590910066880554 L26.364027695597436 14.590924706451233 L27.342284752948 14.590988144585424 L28.320541810298565 14.59115893951622 L29.29879886764913 14.59151907260923 L30.277055924999694 14.592173948016352 L31.255312982350258 14.593252391968747 L32.23357003970082 14.594906651510371 L33.21182709705138 14.59731239243007 L34.19008415440195 14.600668696109153 L35.16834121175251 14.605198054953291 L36.14659826910308 14.611146366037417 L37.12485532645364 14.618782922546643 L38.10311238380421 14.628400402552245 L39.08136944115478 14.64031485461976 L40.05962649850534 14.654865679700094 L41.037883555855906 14.672415608713663 L42.016140613206474 14.69335067519188 L42.994397670557035 14.718080182297513 L43.972654727907596 14.747036663503764 L44.950911785258164 14.780675836166324 L45.92916884260873 14.81947654718158 L46.90742589995929 14.86394070988132 L47.885682957309854 14.91459323127134 L48.86394001466043 14.971981928679668 L49.84219707201099 15.036677434839248 L50.82045412936155 15.10927309038766 L51.79871118671212 15.190384822726287 L52.77696824406269 15.280651010141582 L53.75522530141325 15.38073233005133 L54.733482358763816 15.491311590200098 L55.71173941611438 15.613093541589762 L56.689996473464944 15.746804671895461 L57.66825353081551 15.893192978079242 L58.64651058816607 16.053027716879797 L59.62476764551664 16.22709913182382 L60.60302470286721 16.41621815537019 L61.58128176021777 16.621216084768093 L62.55953881756834 16.842944230182013 L63.537795874918906 17.082273533605814 L64.51605293226947 17.34009415706555 L65.49430998962004 17.61731503858499 L66.4725670469706 17.91486341436513 L67.45082410432116 18.23368430561254 L68.42908116167172 18.574739968431686 L69.4073382190223 18.939009305183276 L70.38559527637285 19.327487235699152 L71.36385233372343 19.741184026736335 L72.34210939107399 20.181124578046365 L73.32036644842455 20.64834766343654 L74.29862350577511 21.143905125199694 L75.27688056312567 21.66886102029582 L76.25513762047625 22.22429071667989 L77.23339467782681 22.811279938182565 L78.21165173517738 23.430923756371556 L79.18990879252794 24.084325527843816 L80.1681658498785 24.772595775428385 L81.14642290722907 25.496851011813646 L82.12467996457963 26.258212504152425 L83.1029370219302 27.057804978245372 L84.08119407928078 27.896755260952318 L85.05945113663134 28.776190859542623 L86.0377081939819 29.697238476757946 L87.01596525133246 30.661022460433855 L87.99422230868302 31.668663186605727 L88.9724793660336 32.72127537511 L89.95073642338416 33.819966336787274 L90.92899348073472 34.96583415149445 L91.9072505380853 36.159965776244974 L92.88550759543585 37.4034350829138 L93.86376465278641 38.697300825072546 L94.84202171013699 40.04260453365572 L95.82027876748755 41.44036834130566 L96.79853582483811 42.8915927353986 L97.77679288218869 44.39725423991894 L98.75504993953925 45.95830302652428 L99.73330699688981 47.575660455326954 L100.71156405424038 49.250216546114245 L101.68982111159094 50.98282738093383 L102.6680781689415 52.77431243918491 L103.64633522629207 54.62545186658531 L104.62459228364264 56.53698367961684 L105.6028493409932 58.509600907301376 L106.58110639834376 60.543948672416974 L107.55936345569434 62.64062121453074 L108.5376205130449 64.80015885750618 L109.51587757039546 67.02304492442926 L110.49413462774604 69.3097026031991 L111.4723916850966 71.66049176633828 L112.45064874244716 74.0757057488971 L113.42890579979773 76.55556808865589 L114.4071628571483 79.1002292331674 L115.38541991449885 81.70976321853004 L116.36367697184942 84.38416432513773 L117.34193402919999 87.12334371601685 L118.32019108655055 89.92712606373337 L119.29844814390111 92.79524617223001 L120.27670520125169 95.72734560034095 L121.25496225860225 98.72296929411974 L122.23321931595281 101.78156223551332 L123.21147637330338 104.90246611531364 L124.18973343065394 108.08491603872005 L125.1679904880045 111.32803727225158 L126.14624754535507 114.63084

### Zdefiniowanie zmiennych

In [93]:
val ileDanych = 500
val ileEpok = 100

In [94]:
val x_siec = MutableList(ileDanych) {
    12 * Random.nextDouble()
}

val y_siec: MutableList<Double> = x_siec.map {
    0.5 * cos(0.2 * it * it) + 0.5 + 0.1 * Random.nextDouble(-1.0, 1.0)
}.toMutableList()

In [95]:
val data = mapOf<String, Any>(
    "x" to x_siec,
    "y" to y_siec
)

val plot = ggplot(data) +
        geomLine(color = "red") { x = "x"; y = "y" } +
        ggtitle("y = 0.5 * cos(0.2 * x²) + 0.5 with noise")

plot

<path d="M24.4075135808963 46.01266179342031 L24.4075135808963 46.01266179342031 L25.335009521171344 48.45898892210437 L25.806148841725296 44.21062582618515 L27.916059888376463 30.320008376163884 L27.9298522356548 45.17566551138003 L28.66001589255334 28.03706649284868 L29.493739159118647 47.76222338003711 L30.712874784019583 28.958358976752436 L32.14499058183566 55.76429665596751 L32.266023493011005 38.15142413461095 L32.46974075270782 29.53619426540024 L34.720406599689674 34.21886081815106 L38.057553880504585 62.93844966245999 L40.87392954014457 30.98218398582759 L41.0905789753359 32.523264606968496 L41.24323995827066 22.794432280436922 L41.47826302209671 28.515435660902227 L41.527581508634015 16.79120437007549 L41.91286743417575 35.581477741900216 L43.07227500664425 49.69431713974106 L43.78958044540633 22.005290503813796 L44.98180766937948 48.622927617124816 L47.60175654684283 14.590909090909122 L48.24912127107183 56.70925219039711 L48.542014487644245 28.26352258847328 L50.10757102318317 57.40534230223943 L50.50044849640339 52.12562566748454 L50.77878335283745 53.472337221726974 L51.46167422668567 61.65112886450842 L51.62822767135346 33.85401755101228 L52.49949627474092 43.7577488900979 L52.688168553489135 38.203467620210006 L52.79151482943598 17.856787101978 L52.88447621018314 62.829996431065524 L55.60835379897051 51.9533864159298 L56.533644531050186 30.74975039573559 L56.82472802125942 22.353047049300983 L59.358606993398894 16.000959666485244 L59.88709568336823 25.082645397731937 L60.76295746833584 26.760115102615828 L63.00245429753433 24.57261239202205 L64.11921147804074 43.31125040027959 L65.52329460773676 28.77297377658485 L66.48296311190231 39.23964915174321 L67.44962514277444 39.419111096323206 L71.14746737768837 62.327124318199935 L71.464276346686 46.937268288562024 L72.64940876921142 20.502804522066697 L72.88198520198878 57.69121030140849 L75.08598290567268 63.926484458425534 L75.82889525080849 47.3541282709478 L76.15039394421247 58.83551176066578 L76.24918770555664 61.8579761897019 L76.33391767592852 45.85294617169703 L77.14724978702917 63.43956909763551 L77.16549456994653 24.39920585280464 L78.80231923257854 62.029501454368216 L81.00543914249049 37.84478924103411 L81.15199410730318 27.861068799983087 L84.2326831469219 74.22233645164877 L85.92439793903934 60.8687838237704 L90.33261587553343 68.41706182225857 L90.37290831654016 52.96221820638297 L91.67865096046732 39.67687199852145 L92.07902478892953 74.48373716368732 L92.56243828342764 43.785677920972006 L94.02501387714332 37.59011977547817 L95.35317464506737 44.55380217598986 L95.67528434087012 48.85331778137814 L96.6800477646824 48.16710951188469 L97.47656659320992 83.0694121044466 L97.92824952282037 67.91573473819787 L99.72804148622639 80.67041821479842 L99.75950381386072 90.82110657358632 L99.92706219962878 87.33320954637645 L100.40174592652309 60.45295637890976 L100.84695855733769 58.571645785800484 L101.46533433567166 76.45247223846044 L103.4763322632907 65.90343136571929 L103.71264923333982 95.98354081340273 L105.33163373580032 85.00152179416668 L106.60907905945652 101.1376119414812 L106.7656314970999 64.99728984395489 L107.74291509945269 81.93411790378354 L108.10674969572523 96.63952267323802 L108.62393469110523 81.09560252631479 L108.68535842737477 84.87382734307755 L110.19950106342058 70.34063220536845 L111.70470067793693 73.04513087010147 L112.41237454077755 88.01118143364266 L113.5831060570496 106.00719446040162 L114.80879436462385 73.58044831706954 L115.62316419494861 79.49864471435535 L116.32137228538157 88.75225990667181 L117.69790173467597 114.26319703404869 L119.57864342413858 107.8178619115933 L119.97580108678629 128.31168013858246 L123.02386316283305 123.17870558175315 L123.33955317101093 103.40369857101896 L124.27286718183885 107.66852333120929 L125.08350895582963 105.74437693314815 L126.0329671712804 142.0295165590386 L127.06107914390459 113.20938140369464 L127.34587299139545 114.4523935652202 L127.61983198143918 121.87395642543285 L128.5859036

### Nałożenie dwóch wykresów

In [96]:
val data_siec = mapOf<String, Any>(
    "x_siec" to x_siec,
    "y_siec" to y_siec
)

val data_clean = mapOf<String, Any>(
    "x" to x,
    "y" to y
)

val plot = ggplot() +
        geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
        geomLine(data = data_clean, color = "green") { x = "x"; y = "y" } +
        ggtitle("Nałożenie wykresów")

plot

<path d="M25.330315005551967 46.01266179342031 L25.330315005551967 46.01266179342031 L26.253690432011602 48.45898892210437 L26.722736658682308 44.21062582618515 L28.823274168221893 30.320008376163884 L28.83700524131558 45.17566551138003 L29.563925057130707 28.03706649284868 L30.39394440641839 47.76222338003711 L31.607663872641833 28.958358976752436 L33.033417321391 55.76429665596751 L33.15391252909945 38.15142413461095 L33.35672475002837 29.53619426540024 L35.597391739534245 34.21886081815106 L38.91971333661562 62.93844966245999 L41.72357690395309 30.98218398582759 L41.93926384760991 32.523264606968496 L42.09124661551219 22.794432280436922 L42.32522556071801 28.515435660902227 L42.37432494386488 16.79120437007549 L42.75789918969984 35.581477741900216 L43.91215595264713 49.69431713974106 L44.626274674527835 22.005290503813796 L45.81320528367587 48.622927617124816 L48.4215147187692 14.590909090909122 L49.066003446269846 56.70925219039711 L49.357595449011654 28.26352258847328 L50.9161968087343 57.40534230223943 L51.30732887587148 52.12562566748454 L51.5844271957651 53.472337221726974 L52.264284243633156 61.65112886450842 L52.43009775431313 33.85401755101228 L53.297495640685796 43.7577488900979 L53.48532971982858 38.203467620210006 L53.588216867342275 17.856787101978 L53.68076525581957 62.829996431065524 L56.39254168626359 51.9533864159298 L57.31372170143417 30.74975039573559 L57.603512017742844 22.353047049300983 L60.12613392356224 16.000959666485244 L60.652274738081395 25.082645397731937 L61.52424540026493 26.760115102615828 L63.75379299171092 24.57261239202205 L64.86558884226329 43.31125040027959 L66.26343416143888 28.77297377658485 L67.21883922125075 39.23964915174321 L68.18120673817401 39.419111096323206 L71.86262085803693 62.327124318199935 L72.17802236455748 46.937268288562024 L73.35788969177838 20.502804522066697 L73.58943287539228 57.69121030140849 L75.7836390505176 63.926484458425534 L76.5232509169679 47.3541282709478 L76.84332131322225 58.83551176066578 L76.94167617123452 61.8579761897019 L77.02602971838087 45.85294617169703 L77.83574850239859 63.43956909763551 L77.85391223064303 24.39920585280464 L79.48346510011359 62.029501454368216 L81.67679738117283 37.84478924103411 L81.82270125768314 27.861068799983087 L84.88970396057789 74.22233645164877 L86.57390310344019 60.8687838237704 L90.96253699420438 68.41706182225857 L91.00265043112859 52.96221820638297 L92.3025921541352 39.67687199852145 L92.70118727304205 74.48373716368732 L93.18245314414244 43.785677920972006 L94.6385310674336 37.59011977547817 L95.96079131918754 44.55380217598986 L96.28147000338797 48.85331778137814 L97.28176964316707 48.16710951188469 L98.07474983966974 83.0694121044466 L98.52442611278626 67.91573473819787 L100.31622228092456 80.67041821479842 L100.3475448333318 90.82110657358632 L100.51435882053866 87.33320954637645 L100.98693370709236 60.45295637890976 L101.43016842650442 58.571645785800484 L102.04579699503715 76.45247223846044 L104.04786081913758 65.90343136571929 L104.28312792223028 95.98354081340273 L105.89491988861636 85.00152179416668 L107.16669000564792 101.1376119414812 L107.32254693999676 64.99728984395489 L108.29548884077414 81.93411790378354 L108.65770705748601 96.63952267323802 L109.17259439545745 81.09560252631479 L109.23374524928848 84.87382734307755 L110.74116112203603 70.34063220536845 L112.2396737037178 73.04513087010147 L112.94420363912755 88.01118143364266 L114.10973403787719 106.00719446040162 L115.32997707568212 73.58044831706954 L116.14072896872976 79.49864471435535 L116.83583518458005 88.75225990667181 L118.20624923393916 114.26319703404869 L120.07863549910488 107.8178619115933 L120.47402874040606 128.31168013858246 L123.5085494289931 123.17870558175315 L123.82283694581379 103.40369857101896 L124.75200459533838 107.66852333120929 L125.55904499439691 105.74437693314815 L126.50428512594668 142.0295165590386 L127.52782958576243 113.20938140369464 L127.81135820185747 114.4523935652202 L128.0841000956808 121.87395642543285 L129.04

In [97]:
val nn = NeuralNetwork(
    inputSize = 500,
    hiddenLayers = listOf(5),
    outputSize = 500
)

In [98]:
nn.train(
    inputs = listOf(x_siec),
    targets = listOf(y_siec),
    epochs = ileEpok,
    learningRate = 0.01
)

val y_pred = nn.predict(x_siec)

In [99]:
val data_siec = mapOf<String, Any>(
    "x_siec" to x_siec,
    "y_siec" to y_siec
)

val data_pred = mapOf<String, Any>(
    "x" to x_siec,
    "y" to y_pred
)

val plot = ggplot() +
        geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
        geomLine(data = data_pred, color = "green") { x = "x"; y = "y" } +
        ggtitle("Nałożenie wykresów")

plot

<path d="M24.4075135808963 46.01266179342031 L24.4075135808963 46.01266179342031 L25.335009521171344 48.45898892210437 L25.806148841725296 44.21062582618515 L27.916059888376463 30.320008376163884 L27.9298522356548 45.17566551138003 L28.66001589255334 28.03706649284868 L29.493739159118647 47.76222338003711 L30.712874784019583 28.958358976752436 L32.14499058183566 55.76429665596751 L32.266023493011005 38.15142413461095 L32.46974075270782 29.53619426540024 L34.720406599689674 34.21886081815106 L38.057553880504585 62.93844966245999 L40.87392954014457 30.98218398582759 L41.0905789753359 32.523264606968496 L41.24323995827066 22.794432280436922 L41.47826302209671 28.515435660902227 L41.527581508634015 16.79120437007549 L41.91286743417575 35.581477741900216 L43.07227500664425 49.69431713974106 L43.78958044540633 22.005290503813796 L44.98180766937948 48.622927617124816 L47.60175654684283 14.590909090909122 L48.24912127107183 56.70925219039711 L48.542014487644245 28.26352258847328 L50.10757102318317 57.40534230223943 L50.50044849640339 52.12562566748454 L50.77878335283745 53.472337221726974 L51.46167422668567 61.65112886450842 L51.62822767135346 33.85401755101228 L52.49949627474092 43.7577488900979 L52.688168553489135 38.203467620210006 L52.79151482943598 17.856787101978 L52.88447621018314 62.829996431065524 L55.60835379897051 51.9533864159298 L56.533644531050186 30.74975039573559 L56.82472802125942 22.353047049300983 L59.358606993398894 16.000959666485244 L59.88709568336823 25.082645397731937 L60.76295746833584 26.760115102615828 L63.00245429753433 24.57261239202205 L64.11921147804074 43.31125040027959 L65.52329460773676 28.77297377658485 L66.48296311190231 39.23964915174321 L67.44962514277444 39.419111096323206 L71.14746737768837 62.327124318199935 L71.464276346686 46.937268288562024 L72.64940876921142 20.502804522066697 L72.88198520198878 57.69121030140849 L75.08598290567268 63.926484458425534 L75.82889525080849 47.3541282709478 L76.15039394421247 58.83551176066578 L76.24918770555664 61.8579761897019 L76.33391767592852 45.85294617169703 L77.14724978702917 63.43956909763551 L77.16549456994653 24.39920585280464 L78.80231923257854 62.029501454368216 L81.00543914249049 37.84478924103411 L81.15199410730318 27.861068799983087 L84.2326831469219 74.22233645164877 L85.92439793903934 60.8687838237704 L90.33261587553343 68.41706182225857 L90.37290831654016 52.96221820638297 L91.67865096046732 39.67687199852145 L92.07902478892953 74.48373716368732 L92.56243828342764 43.785677920972006 L94.02501387714332 37.59011977547817 L95.35317464506737 44.55380217598986 L95.67528434087012 48.85331778137814 L96.6800477646824 48.16710951188469 L97.47656659320992 83.0694121044466 L97.92824952282037 67.91573473819787 L99.72804148622639 80.67041821479842 L99.75950381386072 90.82110657358632 L99.92706219962878 87.33320954637645 L100.40174592652309 60.45295637890976 L100.84695855733769 58.571645785800484 L101.46533433567166 76.45247223846044 L103.4763322632907 65.90343136571929 L103.71264923333982 95.98354081340273 L105.33163373580032 85.00152179416668 L106.60907905945652 101.1376119414812 L106.7656314970999 64.99728984395489 L107.74291509945269 81.93411790378354 L108.10674969572523 96.63952267323802 L108.62393469110523 81.09560252631479 L108.68535842737477 84.87382734307755 L110.19950106342058 70.34063220536845 L111.70470067793693 73.04513087010147 L112.41237454077755 88.01118143364266 L113.5831060570496 106.00719446040162 L114.80879436462385 73.58044831706954 L115.62316419494861 79.49864471435535 L116.32137228538157 88.75225990667181 L117.69790173467597 114.26319703404869 L119.57864342413858 107.8178619115933 L119.97580108678629 128.31168013858246 L123.02386316283305 123.17870558175315 L123.33955317101093 103.40369857101896 L124.27286718183885 107.66852333120929 L125.08350895582963 105.74437693314815 L126.0329671712804 142.0295165590386 L127.06107914390459 113.20938140369464 L127.34587299139545 114.4523935652202 L127.61983198143918 121.87395642543285 L128.5859036

Jak widać ogólny kształt wykresu jest zachowany, ale nie jest on idealny, a właściwie nie jest nawet bliski idealnemu - w kolejnym kroku porównanie dla różnych ilości epok i warstw

In [111]:
import org.jetbrains.letsPlot.core.spec.plotson.plot

val params: Map<String, List<Any>> = mapOf(
    "epoki" to listOf(10, 50, 100, 200, 250, 500),
    "warstwy" to listOf(5, 4, 2, 1, 5, 1),
    "learningRate" to listOf(0.1, 0.01, 0.1, 0.25, 0.15, 0.1),
)

// val plots: MutableList<Figure?> = mutableListOf()

for (current_iteration in 0 until params["epoki"]!!.size) {
    val current_epochs = params["epoki"]!![current_iteration] as Int
    val current_layers = params["warstwy"]!![current_iteration] as Int
    val current_learningRate = params["learningRate"]!![current_iteration] as Double

    val nn = NeuralNetwork(
        inputSize = 500,
        hiddenLayers = List(current_layers) { 5 },
        outputSize = 500
    )

    nn.train(
        inputs = listOf(x_siec),
        targets = listOf(y_siec),
        epochs = current_epochs,
        learningRate = current_learningRate,
        validationInputs = listOf(x),
        validationTargets = listOf(y)
    )

    val y_pred = nn.predict(x)

    val data_siec = mapOf<String, Any>(
        "x_siec" to x_siec,
        "y_siec" to y_siec
    )

    val data_pred = mapOf<String, Any>(
        "x" to x_siec,
        "y" to y_pred
    )

    val plot = ggplot() +
            geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
            geomLine(data = data_pred, color = "green") { x = "x"; y = "y" } +
            ggtitle("epoki: $current_epochs, warstwy: $current_layers, lr: $current_learningRate")

    //plots.add(plot)
    val epochsRange = (0 until current_epochs).toList()
    val errorData = mapOf(
        "epoch" to epochsRange + epochsRange,
        "error" to nn.trainErrors + nn.validationErrors,
        "type" to List(current_epochs) { "Train Error" } + List(current_epochs) { "Validation Error" }
    )

    val errorPlot = ggplot(errorData) +
            geomLine(color = null) {
                x = "epoch"
                y = "error"
                color = "type"  // map color to the "type" variable
            } +
            ggtitle("epoki: $current_epochs, warstwy: $current_layers, lr: $current_learningRate") +
            scaleColorManual(values = mapOf("Train Error" to "blue", "Validation Error" to "red"))

    gggrid(listOf(plot, errorPlot), ncol = 2).show()
}

// gggrid(plots, ncol = 3).show()

Jak widać w tym przypadku największy wpływ na dopasowanie wykresu miało learning rate ;), dodatkowo ilość epok miało pewien wpływ, ale nie tak znaczący jak learning rate, a ilość warstw nie miała praktycznie żadnego wpływu na dopasowanie wykresu - widać to doskonale w porównaniu figur 1 i 5

<<TO DO>> WYKRES BLEDU